In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brist1d/sample_submission.csv
/kaggle/input/brist1d/activities.txt
/kaggle/input/brist1d/train.csv
/kaggle/input/brist1d/test.csv


In [2]:
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from lightgbm import early_stopping

import warnings

warnings.filterwarnings('ignore')

In [3]:
train=pd.read_csv('/kaggle/input/brist1d/train.csv')
test=pd.read_csv('/kaggle/input/brist1d/test.csv')
train.shape,test.shape

((177024, 508), (3644, 507))

In [4]:
train.head(10)

,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
0,p01_0,p01,06:10:00,NaN,NaN,9.6,NaN,NaN,9.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
1,p01_1,p01,06:25:00,NaN,NaN,9.7,NaN,NaN,9.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
2,p01_2,p01,06:40:00,NaN,NaN,9.2,NaN,NaN,8.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
3,p01_3,p01,06:55:00,NaN,NaN,8.7,NaN,NaN,8.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
4,p01_4,p01,07:10:00,NaN,NaN,8.4,NaN,NaN,8.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7
5,p01_5,p01,07:25:00,NaN,NaN,8.1,NaN,NaN,8.3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.4
6,p01_6,p01,07:40:00,NaN,NaN,8.3,NaN,NaN,9.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.9
7,p01_7,p01,07:55:00,NaN,NaN,9.6,NaN,NaN,11.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.1
8,p01_8,p01,08:10:00,NaN,NaN,11.1,NaN,NaN,11.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.1
9,p01_9,p01,08:25:00,NaN,NaN,11.8,NaN,NaN,12.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.9


In [5]:
train.columns

Index(['id', 'p_num', 'time', 'bg-5:55', 'bg-5:50', 'bg-5:45', 'bg-5:40',
       'bg-5:35', 'bg-5:30', 'bg-5:25',
       ...
       'activity-0:40', 'activity-0:35', 'activity-0:30', 'activity-0:25',
       'activity-0:20', 'activity-0:15', 'activity-0:10', 'activity-0:05',
       'activity-0:00', 'bg+1:00'],
      dtype='object', length=508)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177024 entries, 0 to 177023
Columns: 508 entries, id to bg+1:00
dtypes: float64(433), object(75)
memory usage: 686.1+ MB


In [7]:
train.describe()

,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,bg-5:20,bg-5:15,bg-5:10,...,cals-0:40,cals-0:35,cals-0:30,cals-0:25,cals-0:20,cals-0:15,cals-0:10,cals-0:05,cals-0:00,bg+1:00
count,149770.000000,158533.000000,163364.000000,149766.000000,158254.000000,163770.000000,149763.000000,157973.000000,164174.000000,149771.000000,...,141694.000000,141706.000000,141713.000000,141722.000000,141732.000000,141741.000000,141751.000000,141761.000000,141767.000000,177024.000000
mean,8.211018,8.230449,8.253291,8.210988,8.229649,8.254083,8.211049,8.228888,8.254602,8.211000,...,9.364063,9.364096,9.362525,9.366591,9.368108,9.366998,9.370010,9.372536,9.368960,8.277045
std,2.852188,2.913438,2.945594,2.852090,2.911313,2.947651,2.852212,2.909304,2.949689,2.852315,...,8.012264,8.012681,8.017508,8.018465,8.018349,8.023788,8.021999,8.023280,8.023327,2.996398
min,2.200000,2.200000,2.200000,2.200000,2.200000,2.200000,2.200000,2.200000,2.200000,2.200000,...,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,2.200000
25%,6.100000,6.100000,6.100000,6.100000,6.100000,6.100000,6.100000,6.100000,6.100000,6.100000,...,5.600000,5.600000,5.600000,5.600000,5.600000,5.600000,5.600000,5.600000,5.600000,6.100000
50%,7.600000,7.600000,7.700000,7.600000,7.600000,7.700000,7.600000,7.600000,7.700000,7.600000,...,6.180000,6.180000,6.170000,6.180000,6.180000,6.180000,6.180000,6.180000,6.180000,7.700000
75%,9.800000,9.800000,9.800000,9.800000,9.800000,9.800000,9.800000,9.800000,9.800000,9.800000,...,9.060000,9.060000,9.050000,9.060000,9.060000,9.060000,9.060000,9.070000,9.060000,9.900000
max,22.200000,25.100000,27.800000,22.200000,25.100000,27.800000,22.200000,25.100000,27.800000,22.200000,...,116.100000,116.100000,116.100000,116.100000,116.100000,116.100000,116.100000,116.100000,116.100000,27.800000


In [8]:
train.isnull().sum()

id                    0
p_num                 0
time                  0
bg-5:55           27254
bg-5:50           18491
                  ...  
activity-0:15    174293
activity-0:10    174287
activity-0:05    174271
activity-0:00    174287
bg+1:00               0
Length: 508, dtype: int64

In [9]:
# Create a mask of columns to drop in the training set
train_missing = train.isnull().mean()*100

# Identify columns with more than 20% missing data in train
columns_to_drop = train_missing[train_missing > 25].index

In [10]:
columns_to_drop.shape

(288,)

In [11]:
# Drop the identified columns from both df_train and df_test
train_cleaned = train.drop(columns=columns_to_drop)
test_cleaned = test.drop(columns=columns_to_drop)


train_cleaned.shape,test_cleaned.shape

((177024, 220), (3644, 219))

In [12]:
# Check the result
print(f"Columns dropped: {columns_to_drop}")
print("Updated df_train:",train_cleaned.shape)
print("Updated df_test:",test_cleaned.shape)

Columns dropped: Index(['carbs-5:55', 'carbs-5:50', 'carbs-5:45', 'carbs-5:40', 'carbs-5:35',
       'carbs-5:30', 'carbs-5:25', 'carbs-5:20', 'carbs-5:15', 'carbs-5:10',
       ...
       'activity-0:45', 'activity-0:40', 'activity-0:35', 'activity-0:30',
       'activity-0:25', 'activity-0:20', 'activity-0:15', 'activity-0:10',
       'activity-0:05', 'activity-0:00'],
      dtype='object', length=288)
Updated df_train: (177024, 220)
Updated df_test: (3644, 219)


In [13]:
# Drop the 'id' column from both datasets
train_cleaned = train_cleaned.drop('id', axis=1)
test_cleaned = test_cleaned.drop('id', axis=1)

In [14]:
# Convert time columns to hours, minutes, and seconds
def convert_time_columns(train_cleaned):
    for col in train_cleaned.columns:
        if pd.api.types.is_datetime64_any_dtype(train_cleaned[col]) or 'time' in col.lower():
            train_cleaned[col] = pd.to_datetime(train_cleaned[col], errors='coerce')  # Convert to datetime
            train_cleaned[col+'_hour'] = train_cleaned[col].dt.hour
            train_cleaned[col+'_minute'] = train_cleaned[col].dt.minute
#             train_cleaned[col+'_second'] = train_cleaned[col].dt.second
            train_cleaned.drop(col, axis=1, inplace=True)  # Drop original time column after conversion
# Apply time conversion to both train and test datasets
convert_time_columns(train_cleaned)
convert_time_columns(test_cleaned)

In [15]:
# Handle 'p_num' in the test set: find unique values and replace them with numbers
unique_pnums = test_cleaned['p_num'].unique()
pnum_mapping = {value: idx for idx, value in enumerate(unique_pnums)}
test_cleaned['p_num'] = test_cleaned['p_num'].map(pnum_mapping)

# Also ensure the same mapping is applied to 'p_num' in the train set, if it exists
if 'p_num' in train_cleaned.columns:
    train_cleaned['p_num'] = train_cleaned['p_num'].map(pnum_mapping)

In [16]:
print(train_cleaned.shape)
print(test_cleaned.shape)

(177024, 220)
(3644, 219)


In [17]:
from sklearn.impute import SimpleImputer

# Use mean (you can also try mean or most_frequent based on your data)
imputer = SimpleImputer(strategy='mean')

# Fit on df_train and transform both df_train and df_test
train_imputed = imputer.fit_transform(train_cleaned)
test_imputed = imputer.fit_transform(test_cleaned)

# Convert the NumPy arrays back to DataFrames and retain the original column names
train_cleaned = pd.DataFrame(train_imputed, columns=train_cleaned.columns)
test_cleaned = pd.DataFrame(test_imputed, columns=test_cleaned.columns)


train_float_columns = train_cleaned.select_dtypes(include=['float64', 'float32']).columns.tolist()
test_float_columns = test_cleaned.select_dtypes(include=['float64', 'float32']).columns.tolist()

train_cleaned[train_float_columns] = train_cleaned[train_float_columns].astype(float)
test_cleaned[test_float_columns] = test_cleaned[test_float_columns].astype(float)

print(train_cleaned.dtypes)
print(train_cleaned.columns)


p_num          float64
bg-5:55        float64
bg-5:50        float64
bg-5:45        float64
bg-5:40        float64
                ...   
cals-0:05      float64
cals-0:00      float64
bg+1:00        float64
time_hour      float64
time_minute    float64
Length: 220, dtype: object
Index(['p_num', 'bg-5:55', 'bg-5:50', 'bg-5:45', 'bg-5:40', 'bg-5:35',
       'bg-5:30', 'bg-5:25', 'bg-5:20', 'bg-5:15',
       ...
       'cals-0:30', 'cals-0:25', 'cals-0:20', 'cals-0:15', 'cals-0:10',
       'cals-0:05', 'cals-0:00', 'bg+1:00', 'time_hour', 'time_minute'],
      dtype='object', length=220)


In [18]:
train_cleaned.isnull().sum()

p_num          0
bg-5:55        0
bg-5:50        0
bg-5:45        0
bg-5:40        0
              ..
cals-0:05      0
cals-0:00      0
bg+1:00        0
time_hour      0
time_minute    0
Length: 220, dtype: int64

In [19]:
test_cleaned.isnull().sum()

p_num          0
bg-5:55        0
bg-5:50        0
bg-5:45        0
bg-5:40        0
              ..
cals-0:10      0
cals-0:05      0
cals-0:00      0
time_hour      0
time_minute    0
Length: 219, dtype: int64

In [20]:
# Prepare Features and Target Variable for Training Data

x_train = train_cleaned.drop('bg+1:00', axis=1)
y_train = train_cleaned['bg+1:00']

In [21]:
x_train_split, x_val, y_train_split, y_val = train_test_split(x_train, y_train, test_size = 0.2 , random_state = 42)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler, PowerTransformer
from scipy import sparse
import numpy as np
import optuna

# Reduce memory usage for dataframes
def reduce_memory_usage(df):
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object and col_type.name != 'category':
            if col_type.kind in 'i':  # Integer
                df[col] = pd.to_numeric(df[col], downcast='integer')
            elif col_type.kind in 'f':  # Float
                df[col] = pd.to_numeric(df[col], downcast='float')
    return df

# Sanitize feature names
def sanitize_feature_names(df):
    return df.rename(columns=lambda x: x.replace("+", "_plus_")
                                    .replace(":", "_colon_")
                                    .replace("-", "_dash_")
                                    .replace(" ", "_"))

# Apply to all datasets
x_train_split = reduce_memory_usage(sanitize_feature_names(x_train_split))
x_val = reduce_memory_usage(sanitize_feature_names(x_val))
x_test = reduce_memory_usage(sanitize_feature_names(test_cleaned)).drop('bg_plus_1_00', axis=1, errors='ignore')

# Ensure the target (y_train_split) matches x_train_split's index
y_train_split = y_train_split.loc[x_train_split.index]

# Feature engineering: Add polynomial interaction features
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
x_train_poly = sparse.csr_matrix(poly.fit_transform(x_train_split))
x_val_poly = sparse.csr_matrix(poly.transform(x_val))
x_test_poly = sparse.csr_matrix(poly.transform(x_test))

# Power transform features for normalization
scaler = StandardScaler()
power_transformer = PowerTransformer()

x_train_scaled = power_transformer.fit_transform(scaler.fit_transform(x_train_poly.toarray()))
x_val_scaled = power_transformer.transform(scaler.transform(x_val_poly.toarray()))
x_test_scaled = power_transformer.transform(scaler.transform(x_test_poly.toarray()))

# Sample a subset of data for hyperparameter tuning
x_train_sample = sparse.csr_matrix(x_train_scaled[:len(x_train_scaled) // 2])
y_train_sample = y_train_split.iloc[:len(x_train_split) // 2]

# Define the objective function for Optuna
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-4, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 10, 150),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-3, 10.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-3, 10.0),
        "device": "gpu",
        "gpu_id": 0,
    }

    model = LGBMRegressor(**params, random_state=42)
    selector = RFECV(estimator=model, step=10, cv=3, scoring="neg_root_mean_squared_error", n_jobs=-1)
    selector.fit(x_train_sample.toarray(), y_train_sample)  # Use dense data only here

    x_train_selected = x_train_sample[:, selector.support_]
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = []

    for train_idx, valid_idx in kf.split(x_train_selected, y_train_sample):
        x_train_fold, x_valid_fold = x_train_selected[train_idx].toarray(), x_train_selected[valid_idx].toarray()
        y_train_fold, y_valid_fold = y_train_sample.iloc[train_idx], y_train_sample.iloc[valid_idx]

        model.fit(
            x_train_fold, y_train_fold,
            eval_set=[(x_valid_fold, y_valid_fold)],
            eval_metric="rmse",
            callbacks=[early_stopping(stopping_rounds=10)]
        )
        y_pred_fold = model.predict(x_valid_fold)
        fold_rmse = np.sqrt(mean_squared_error(y_valid_fold, y_pred_fold))
        rmse_scores.append(fold_rmse)

    return np.mean(rmse_scores)

# Setup Optuna study
study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=50, n_jobs=-1)

# Print best parameters
print("Best parameters:", study.best_params)
print("Best RMSE:", study.best_value)

# Final model
best_params = study.best_params
final_model = LGBMRegressor(**best_params, random_state=42)

# Train on full dataset with selected features
selector = RFECV(estimator=final_model, step=10, cv=3, scoring="neg_root_mean_squared_error", n_jobs=-1)
selector.fit(x_train_scaled, y_train_split)
selected_features_mask = selector.support_

x_train_selected = x_train_scaled[:, selected_features_mask]
x_val_selected = x_val_scaled[:, selected_features_mask]
x_test_selected = x_test_scaled[:, selected_features_mask]

final_model.fit(x_train_selected, y_train_split)

# Predict and evaluate
y_val_pred = final_model.predict(x_val_selected)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f"Final RMSE on Validation Set: {rmse}")

# Predict on the test set
test_predictions = final_model.predict(x_test_selected)
print("Test Predictions:", test_predictions)

In [ ]:
#Visualize Predictions
plt.figure(figsize=(12, 6))
plt.plot(y_val.values, label='Actual Glucose Levels')
plt.plot(y_pred, label='Predicted Glucose Levels')
plt.xlabel('Samples')
plt.ylabel('Glucose Level')
plt.title('Predicted vs Actual Glucose Levels')
plt.legend()
plt.show()

In [ ]:
# Save predictions to CSV
submission = pd.DataFrame({'id': test['id'], 'bg+1:00': test_predictions})
submission.to_csv(f'/kaggle/working/submission.csv', index=False)

In [ ]:
print("Model training, evaluation, and test predictions completed.")